In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| code-fold: false
# IMPORTING LIBRARIES
# --------------------

# AstroML
from astroML.datasets import fetch_LINEAR_sample
from astropy.timeseries import LombScargle
from astroML.datasets import fetch_LINEAR_sample
from astroML.datasets import fetch_LINEAR_geneva
from astropy.timeseries import TimeSeries
from astropy.table import Table
from astroML.time_series import MultiTermFit

# ZTF
from ztfquery import lightcurve

# Basic libraries
import pickle
import os
import sys
from tqdm import tqdm

# Plotting
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties

# DataFrame analysis
import pandas as pd

# Math libraries
import numpy as np
import scipy as sc
from scipy.stats import norm

# CONFIGURATION
# -------------
sns.set_theme(style='white') # setting the theme for plotting
sys.path.insert(0,'../src/')
np.random.seed(42)

# configuring plotting colors
colors = ['#465BBB', '#3F8FCE', '#7ABBCE', '#3A3865', '#A82F43', '#612A37', '#DC5433', '#F29457']
b1 = '#465BBB'
b2 = '#3F8FCE'
b3 = '#7ABBCE'
b4 = '#3A3865'
black1 = '#22212A'
black2 = '#2D1E21'
or1 = '#A82F43'
or2 = '#612A37'
or3 = '#DC5433'
or4 = '#F29457'
muted_colors = sns.set_palette(sns.color_palette(colors))

# configuring fonts for plotting
font = FontProperties()
font.set_family('serif')
font.set_name('Andale Mono')
font.set_style('normal')
#font.set_size('normal')


%matplotlib inline 

In [3]:
#| code-fold: false
# Importing custom libraries
# ----------------------------
from data_access import*
from helper import*
from RR_lyrae_params import*
from blazhko_analysis import*
from BE_plotting import*

# Visual analysis

The final step of this project is to visually analyze and confirm the algorithms selection of Blazhko candidates. We perform this operation using a custom made interface that displays 4 graphs, letting the user analyze them and click 'KEEP' if it confirms the star is Blazhko, or 'CONTINUE' if it is not.

In [4]:
# DATA
# ======
# LINEAR data
dataL = fetch_LINEAR_sample(data_home='../inputs')

# calcualted data
end = 'ffinal'

# light curve table
lc_analysis = pd.read_csv("../outputs/LC_ANALYSIS_"+end+".csv")
# rr lyrae
rrlyrae = pd.read_csv('../outputs/Lrrlyr_unprocessed.csv')
# blazhko candidates
blazhko_candidates = pd.read_csv('../outputs/blazhko_canmodfinal.csv')
# periodogram data
periodogr = pickle.load(open("../outputs/periodograms_"+end+".pkl", "rb"))
# fits data
fits = np.load("../outputs/fits_"+end+".npy", allow_pickle=True)
# ztf data
ztf_data = np.load("../outputs/ztf_data_filter"+end+".npy", allow_pickle=True)

This process was conducted in 4 phases:
1. ### The phased graph
Firstly, the shape and noisiness of the phased light curves were examined. If the light curve had the correct shape, there wasn't too much noise ($\chi^2$ wasn't too high) and there appeared to be smearing of the phased light curve ( a sign of modulation ), the light curve passed the first phase.

2. ### The periodogram
Secondly, the correctness of the algorithm in recognizing the blazhko frequency was looked at. If the blazhko peaks are right next to the yearly alias and not statistically significant, or if the algorithm detected a false signal, the light curve wouldn't pass this phase.

3. ### The full data
Thirdly, the general shape of the light curve was looked at. If the data showed a wave-like pattern, it would indicate the presence of amplitude modulation.

4. ### Seasons of observation
Finally, the most important phase, where we analyzed the light curve fit for each observation season, comparing how the light curve changed depending on the phased and normalized graph. We look for a translation in the x-coordinate, or a *phase modulation*, or if the width of the light curve changes during seasons, we found *amplitude modulation*.

A star can satisfy all 4 criteria, the first and final stage, or just the second stage to be considered a Blazhko star.

We split up our 409 candidates into 3 categories for more efficient visual analysis:
1. ZTF periodogram group
2. LINEAR periodogram group
3. Other metrics group

## Periodogram group

In [5]:
# ZTF PERIODOGRAM
# =================
ztf_per = category_analysis(blazhko_candidates, fits, periodogr, ztf_data, dataL,id_list=None,parameter='IndicatorType', value='Z')

This dataset has 10 stars.


In [6]:
blazhko_analyzer = ztf_per.get_save_data()
end = 'Z_p'
blazhko_analyzer.to_csv("../outputs/group"+end+".csv", index=False)

print(blazhko_analyzer.shape)

(4, 34)


In [7]:
# LINEAR PERIODOGRAM
# ======================
linear_per = category_analysis(blazhko_candidates, fits, periodogr, ztf_data, dataL,id_list=None,parameter='IndicatorType', value='L')

This dataset has 42 stars.


In [8]:
blazhko_analyzerl = linear_per.get_save_data()
end = 'L_p'
blazhko_analyzerl.to_csv("../outputs/group"+end+".csv", index=False)

print(blazhko_analyzerl.shape)

(10, 34)


## Score group

In [5]:
# OTHER
# =======
be_other = blazhko_candidates[(blazhko_candidates['IndicatorType']!= 'Z') & (blazhko_candidates['IndicatorType']!= 'L')]
be_other = be_other.reset_index(drop=True)
print(be_other.shape)
be_other.head()

(479, 34)


,LINEAR id,Plinear,Pztf,Pmean,Pratio,NdataLINEAR,L_chi2dof,L_chi2dofR,Lmean_chi2dof,Lmean_chi2dofR,...,MainPeakZ,BlazhkoPeakZ,BlazhkoPeriodZ,BpowerRatioZ,BsignificanceZ,Ampl_diff,dP,IndicatorType,ChiType,BE_score
0,158779,0.609207,0.609189,0.609198,0.999970,293,3.7,1.6,4.3,1.6,...,1.6415,1.6444,350.2627,0.0123,16.5371,0.21,0.00003,-,LZ4,7
1,263541,0.558218,0.558221,0.558220,1.000005,270,15.8,2.9,16.2,3.0,...,1.7914,1.8025,89.9685,0.0021,7.5968,0.18,0.00001,-,LZ5,7
2,393084,0.530027,0.530033,0.530030,1.000011,493,1.6,1.1,1.6,1.1,...,1.8867,1.8896,347.2222,0.0235,16.1149,0.35,0.00001,-,LZ4,6
3,514883,0.557723,0.557737,0.557730,1.000025,289,5.3,1.7,5.4,1.6,...,1.7930,1.7958,357.1429,0.0149,17.9476,0.17,0.00003,-,LZ5,8
4,737951,0.357023,0.357023,0.357023,1.000000,273,6.0,2.2,6.0,2.2,...,2.8009,2.8039,332.2259,0.0020,13.7006,0.09,0.00000,-,LZ5,6


In [7]:
ind = be_other.loc[be_other['LINEAR id'] == 12172879]
ind

,LINEAR id,Plinear,Pztf,Pmean,Pratio,NdataLINEAR,L_chi2dof,L_chi2dofR,Lmean_chi2dof,Lmean_chi2dofR,...,MainPeakZ,BlazhkoPeakZ,BlazhkoPeriodZ,BpowerRatioZ,BsignificanceZ,Ampl_diff,dP,IndicatorType,ChiType,BE_score
242,12172879,0.377286,0.377308,0.377297,1.000058,229,2.5,1.5,3.6,1.7,...,2.6504,2.6553,202.8398,0.0014,6.7895,0.11,0.00006,-,LZ4,7


In [11]:
other = list(be_other['LINEAR id'])[243:]
len(other)

236

In [12]:
score = category_analysis(be_other, fits, periodogr, ztf_data, dataL, id_list=other)

This dataset has 236 stars.


In [ ]:
# star 109, 125 is an amazing example of blazhko

In [14]:
blazhko_analyzerother = score.get_save_data()
end = 'score_2'
blazhko_analyzerother.to_csv("../outputs/group_"+end+".csv", index=False)

print(blazhko_analyzerother.shape)

(93, 34)


---

In [15]:
bL = pd.read_csv('../outputs/groupL_p.csv')
bZ = pd.read_csv('../outputs/groupZ_p.csv')
bo1 = pd.read_csv('../outputs/group_score.csv')
bo2 = pd.read_csv('../outputs/group_score_2.csv')

In [16]:
FINAL_BE_CATALOGUE = pd.DataFrame()
FINAL_BE_CATALOGUE = pd.concat([FINAL_BE_CATALOGUE, bL.reset_index(drop=True)],ignore_index=True, axis=0)
FINAL_BE_CATALOGUE = pd.concat([FINAL_BE_CATALOGUE, bZ.reset_index(drop=True)],ignore_index=True, axis=0)
FINAL_BE_CATALOGUE = pd.concat([FINAL_BE_CATALOGUE, bo1.reset_index(drop=True)],ignore_index=True, axis=0)
FINAL_BE_CATALOGUE = pd.concat([FINAL_BE_CATALOGUE, bo2.reset_index(drop=True)],ignore_index=True, axis=0)

print(FINAL_BE_CATALOGUE.shape)
FINAL_BE_CATALOGUE.head()

(228, 34)


,LINEAR id,Plinear,Pztf,Pmean,Pratio,NdataLINEAR,L_chi2dof,L_chi2dofR,Lmean_chi2dof,Lmean_chi2dofR,...,MainPeakZ,BlazhkoPeakZ,BlazhkoPeriodZ,BpowerRatioZ,BsignificanceZ,Ampl_diff,dP,IndicatorType,ChiType,BE_score
0,3632428,0.710123,0.710081,0.710102,0.999941,216,1.1,0.9,1.3,1.1,...,1.4083,1.4148,153.9646,0.0025,9.7161,0.15,0.00006,L,-,0
1,7545954,0.555952,0.555959,0.555956,1.000013,228,29.7,3.7,28.4,3.8,...,1.7987,1.8021,296.2963,0.0031,9.2400,0.04,0.00001,L,-,0
2,8955278,0.547521,0.547514,0.547518,0.999987,200,3.8,1.3,4.1,1.3,...,1.8264,1.8402,72.7273,0.0095,10.8462,0.01,0.00001,L,-,0
3,13331395,0.646706,0.646719,0.646712,1.000020,194,5.4,1.2,5.5,1.2,...,1.5463,1.5572,91.4913,0.0042,17.7771,0.15,0.00002,L,-,0
4,14225011,0.610380,0.610382,0.610381,1.000003,167,1.2,1.0,1.2,1.0,...,1.6383,1.6436,187.9699,0.0005,6.4955,0.23,0.00000,L,-,0


In [17]:
FINAL_BE_CATALOGUE.to_csv('../outputs/be_final.csv', index=False)